In [1]:
import pandas as pd
import requests
import json
import csv

from empath import Empath
import liwcanalysis

import re
from collections import Counter
import liwc
import collections

In [39]:
def get_same_topics(list_1, list_2, topN=50):
    # find the correct topN
    if len(list_1) > len(list_2):
        small_list_length = len(list_2)
    else:
        small_list_length = len(list_1)
    if topN > small_list_length:
        topN = small_list_length
        
#     internal_list_1 = [category for (category, value) in list_1[:topN]]
#     internal_list_2 = [category for (category, value) in list_2[:topN]]
#     intersection = set(internal_list_1).intersection(set(internal_list_2))
    
    intersection = []
    for (category_1, value_1) in list_1[:topN]:
        for (category_2, value_2) in list_2[:topN]:
            if category_1 == category_2:
                intersection.append((category_1, value_1+value_2))
    intersection = sorted(intersection, key=lambda elem:elem[1], reverse=True)
    
    print('- topN: ', topN)
    print('  - intersection: ', intersection)

## Empath

In [15]:
# reddit
proposal_empath_result_reddit = [('eating', 237.0), ('negative_emotion', 226.0), ('smell', 219.0), ('communication', 217.0), ('speaking', 191.0), ('business', 190.0), ('economics', 171.0), ('shopping', 168.0), ('liquid', 129.0), ('internet', 122.0), ('real_estate', 119.0), ('alcohol', 119.0), ('money', 118.0), ('positive_emotion', 112.0), ('children', 105.0), ('giving', 97.0), ('valuable', 96.0), ('work', 94.0), ('trust', 93.0), ('violence', 91.0), ('payment', 91.0), ('law', 90.0), ('social_media', 89.0), ('messaging', 89.0), ('leader', 83.0), ('writing', 82.0), ('dispute', 79.0), ('swearing_terms', 78.0), ('strength', 77.0), ('optimism', 74.0), ('traveling', 74.0), ('reading', 71.0), ('healing', 70.0), ('government', 69.0), ('gain', 68.0), ('meeting', 66.0), ('youth', 66.0), ('banking', 63.0), ('hearing', 63.0), ('listen', 63.0), ('shape_and_size', 63.0), ('party', 62.0), ('phone', 60.0), ('help', 59.0), ('toy', 59.0), ('pain', 59.0), ('attractive', 56.0), ('computer', 56.0), ('body', 54.0), ('appearance', 54.0), ('childish', 52.0), ('suffering', 51.0), ('shame', 51.0), ('cooking', 50.0), ('friends', 50.0), ('hate', 49.0), ('office', 48.0), ('family', 48.0), ('fight', 48.0), ('love', 47.0), ('competing', 47.0), ('home', 46.0), ('movement', 46.0), ('injury', 45.0), ('politics', 44.0), ('celebration', 43.0), ('cold', 42.0), ('order', 42.0), ('restaurant', 41.0), ('emotional', 41.0), ('negotiate', 40.0), ('health', 39.0), ('dominant_heirarchical', 39.0), ('technology', 39.0), ('power', 39.0), ('confusion', 38.0), ('programming', 38.0), ('music', 38.0), ('urban', 38.0), ('breaking', 38.0), ('aggression', 37.0), ('wealthy', 37.0), ('play', 37.0), ('driving', 37.0), ('journalism', 36.0), ('college', 36.0), ('wedding', 35.0), ('weather', 35.0), ('war', 35.0), ('nervousness', 34.0), ('leisure', 34.0), ('plant', 34.0), ('school', 32.0), ('vacation', 30.0), ('sports', 30.0), ('heroic', 30.0), ('feminine', 30.0), ('medical_emergency', 29.0), ('kill', 29.0), ('achievement', 29.0), ('death', 28.0), ('vehicle', 27.0), ('sound', 27.0), ('sadness', 27.0), ('fun', 27.0), ('noise', 26.0), ('occupation', 25.0), ('contentment', 25.0), ('crime', 24.0), ('hipster', 24.0), ('stealing', 23.0), ('fashion', 23.0), ('rural', 23.0), ('sleep', 22.0), ('masculine', 22.0), ('ridicule', 22.0), ('hiking', 22.0), ('fire', 22.0), ('clothing', 22.0), ('poor', 22.0), ('deception', 21.0), ('weapon', 21.0), ('white_collar_job', 20.0), ('divine', 19.0), ('military', 19.0), ('car', 19.0), ('cleaning', 19.0), ('ocean', 19.0), ('art', 17.0), ('fabric', 17.0), ('domestic_work', 16.0), ('weakness', 15.0), ('morning', 15.0), ('fear', 15.0), ('legend', 15.0), ('ancient', 15.0), ('affection', 15.0), ('beach', 14.0), ('water', 14.0), ('politeness', 14.0), ('sailing', 14.0), ('envy', 13.0), ('magic', 13.0), ('dominant_personality', 13.0), ('science', 13.0), ('prison', 12.0), ('surprise', 12.0), ('disgust', 12.0), ('tool', 12.0), ('anger', 12.0), ('beauty', 12.0), ('cheerfulness', 11.0), ('royalty', 11.0), ('furniture', 11.0), ('night', 11.0), ('philosophy', 11.0), ('horror', 10.0), ('irritability', 10.0), ('independence', 10.0), ('neglect', 10.0), ('warmth', 10.0), ('musical', 10.0), ('blue_collar_job', 9.0), ('sexual', 9.0), ('sympathy', 9.0), ('ugliness', 9.0), ('lust', 9.0), ('disappointment', 9.0), ('swimming', 8.0), ('animal', 8.0), ('timidity', 8.0), ('exercise', 7.0), ('superhero', 7.0), ('air_travel', 7.0), ('tourism', 6.0), ('torment', 6.0), ('ship', 6.0), ('terrorism', 6.0), ('dance', 5.0), ('pride', 5.0), ('pet', 5.0), ('hygiene', 5.0), ('monster', 5.0), ('exasperation', 4.0), ('zest', 4.0), ('exotic', 4.0), ('joy', 4.0), ('religion', 3.0), ('farming', 3.0), ('rage', 3.0), ('anticipation', 2.0), ('worship', 2.0), ('medieval', 1.0), ('anonymity', 1.0)]
approval_empath_result_reddit = [('negative_emotion', 196.0), ('communication', 148.0), ('business', 139.0), ('economics', 133.0), ('speaking', 131.0), ('smell', 126.0), ('shopping', 119.0), ('eating', 107.0), ('law', 101.0), ('money', 97.0), ('government', 91.0), ('real_estate', 91.0), ('positive_emotion', 88.0), ('dispute', 84.0), ('giving', 84.0), ('valuable', 83.0), ('shape_and_size', 77.0), ('strength', 73.0), ('liquid', 72.0), ('fight', 70.0), ('violence', 69.0), ('trust', 68.0), ('internet', 62.0), ('banking', 61.0), ('leader', 61.0), ('traveling', 60.0), ('reading', 58.0), ('listen', 57.0), ('gain', 57.0), ('children', 57.0), ('messaging', 55.0), ('optimism', 54.0), ('work', 53.0), ('hearing', 52.0), ('swearing_terms', 51.0), ('computer', 51.0), ('competing', 49.0), ('phone', 48.0), ('social_media', 47.0), ('movement', 47.0), ('war', 47.0), ('politics', 46.0), ('power', 46.0), ('hate', 45.0), ('party', 45.0), ('writing', 45.0), ('order', 44.0), ('youth', 44.0), ('pain', 44.0), ('help', 43.0), ('technology', 43.0), ('payment', 43.0), ('healing', 42.0), ('toy', 42.0), ('wedding', 39.0), ('emotional', 39.0), ('weapon', 39.0), ('love', 38.0), ('shame', 38.0), ('crime', 37.0), ('attractive', 37.0), ('stealing', 36.0), ('celebration', 36.0), ('suffering', 35.0), ('appearance', 35.0), ('friends', 35.0), ('school', 34.0), ('sports', 34.0), ('deception', 34.0), ('injury', 34.0), ('alcohol', 34.0), ('leisure', 33.0), ('wealthy', 33.0), ('childish', 33.0), ('heroic', 33.0), ('dominant_heirarchical', 32.0), ('fun', 32.0), ('vacation', 31.0), ('weather', 31.0), ('breaking', 31.0), ('negotiate', 31.0), ('achievement', 31.0), ('restaurant', 30.0), ('meeting', 30.0), ('cold', 29.0), ('journalism', 29.0), ('divine', 29.0), ('body', 29.0), ('music', 29.0), ('sadness', 29.0), ('play', 27.0), ('family', 26.0), ('vehicle', 25.0), ('aggression', 24.0), ('driving', 24.0), ('noise', 24.0), ('confusion', 24.0), ('fire', 24.0), ('health', 23.0), ('art', 22.0), ('college', 22.0), ('urban', 22.0), ('feminine', 22.0), ('ocean', 22.0), ('sleep', 21.0), ('medical_emergency', 21.0), ('occupation', 21.0), ('kill', 21.0), ('home', 21.0), ('fear', 21.0), ('military', 21.0), ('rural', 21.0), ('masculine', 20.0), ('nervousness', 20.0), ('poor', 20.0), ('beach', 19.0), ('cooking', 19.0), ('ancient', 19.0), ('sound', 19.0), ('ridicule', 18.0), ('programming', 18.0), ('sailing', 18.0), ('science', 18.0), ('contentment', 17.0), ('musical', 17.0), ('prison', 16.0), ('affection', 16.0), ('terrorism', 16.0), ('office', 15.0), ('cleaning', 15.0), ('domestic_work', 14.0), ('death', 14.0), ('ship', 14.0), ('plant', 14.0), ('independence', 13.0), ('philosophy', 13.0), ('morning', 12.0), ('swimming', 12.0), ('hiking', 12.0), ('sympathy', 12.0), ('torment', 12.0), ('car', 12.0), ('white_collar_job', 12.0), ('animal', 12.0), ('exercise', 11.0), ('night', 11.0), ('worship', 11.0), ('warmth', 11.0), ('disappointment', 11.0), ('religion', 10.0), ('water', 10.0), ('tool', 10.0), ('ugliness', 10.0), ('monster', 10.0), ('dance', 9.0), ('cheerfulness', 9.0), ('irritability', 9.0), ('surprise', 9.0), ('legend', 9.0), ('air_travel', 9.0), ('joy', 9.0), ('fashion', 9.0), ('lust', 9.0), ('clothing', 9.0), ('horror', 8.0), ('royalty', 8.0), ('hipster', 8.0), ('anger', 8.0), ('beauty', 8.0), ('sexual', 7.0), ('pet', 7.0), ('zest', 7.0), ('fabric', 7.0), ('anticipation', 6.0), ('politeness', 6.0), ('rage', 6.0), ('envy', 5.0), ('tourism', 5.0), ('dominant_personality', 5.0), ('disgust', 5.0), ('weakness', 4.0), ('hygiene', 4.0), ('pride', 3.0), ('furniture', 3.0), ('magic', 3.0), ('neglect', 3.0), ('exotic', 3.0), ('anonymity', 3.0), ('superhero', 2.0), ('exasperation', 2.0), ('blue_collar_job', 1.0), ('farming', 1.0), ('medieval', 0.0), ('timidity', 0.0)]
enforcement_empath_result_01_reddit = [('negative_emotion', 47.0), ('money', 32.0), ('speaking', 31.0), ('business', 30.0), ('communication', 29.0), ('economics', 28.0), ('shopping', 26.0), ('smell', 26.0), ('valuable', 25.0), ('youth', 25.0), ('payment', 25.0), ('children', 24.0), ('law', 23.0), ('real_estate', 20.0), ('violence', 20.0), ('giving', 19.0), ('positive_emotion', 19.0), ('government', 18.0), ('healing', 17.0), ('liquid', 17.0), ('school', 16.0), ('banking', 16.0), ('shape_and_size', 16.0), ('alcohol', 16.0), ('health', 15.0), ('internet', 15.0), ('reading', 15.0), ('trust', 15.0), ('family', 14.0), ('swearing_terms', 14.0), ('college', 14.0), ('home', 14.0), ('eating', 14.0), ('pain', 13.0), ('dispute', 12.0), ('wealthy', 12.0), ('writing', 12.0), ('medical_emergency', 11.0), ('listen', 11.0), ('phone', 11.0), ('messaging', 11.0), ('cold', 10.0), ('hearing', 10.0), ('science', 10.0), ('work', 10.0), ('technology', 10.0), ('domestic_work', 9.0), ('social_media', 9.0), ('childish', 9.0), ('strength', 9.0), ('help', 8.0), ('crime', 8.0), ('suffering', 8.0), ('optimism', 8.0), ('stealing', 8.0), ('movement', 8.0), ('gain', 8.0), ('emotional', 8.0), ('traveling', 8.0), ('shame', 8.0), ('party', 8.0), ('friends', 8.0), ('journalism', 7.0), ('driving', 7.0), ('leader', 7.0), ('body', 7.0), ('order', 7.0), ('injury', 7.0), ('appearance', 7.0), ('politics', 7.0), ('power', 7.0), ('poor', 7.0), ('negotiate', 7.0), ('office', 6.0), ('vacation', 6.0), ('computer', 6.0), ('confusion', 6.0), ('heroic', 6.0), ('love', 6.0), ('ancient', 6.0), ('fight', 6.0), ('toy', 6.0), ('meeting', 6.0), ('white_collar_job', 6.0), ('occupation', 5.0), ('prison', 5.0), ('nervousness', 5.0), ('art', 5.0), ('sports', 5.0), ('death', 5.0), ('celebration', 5.0), ('programming', 5.0), ('dominant_heirarchical', 5.0), ('weather', 5.0), ('breaking', 5.0), ('competing', 5.0), ('contentment', 5.0), ('wedding', 4.0), ('hate', 4.0), ('aggression', 4.0), ('attractive', 4.0), ('divine', 4.0), ('restaurant', 4.0), ('deception', 4.0), ('war', 4.0), ('urban', 4.0), ('tool', 4.0), ('cleaning', 4.0), ('masculine', 3.0), ('leisure', 3.0), ('furniture', 3.0), ('magic', 3.0), ('ridicule', 3.0), ('play', 3.0), ('fear', 3.0), ('irritability', 3.0), ('cooking', 3.0), ('surprise', 3.0), ('noise', 3.0), ('music', 3.0), ('vehicle', 3.0), ('animal', 3.0), ('philosophy', 3.0), ('achievement', 3.0), ('weapon', 3.0), ('musical', 3.0), ('dance', 2.0), ('morning', 2.0), ('exercise', 2.0), ('kill', 2.0), ('sexual', 2.0), ('exasperation', 2.0), ('independence', 2.0), ('fabric', 2.0), ('sound', 2.0), ('sadness', 2.0), ('fashion', 2.0), ('lust', 2.0), ('torment', 2.0), ('car', 2.0), ('terrorism', 2.0), ('plant', 2.0), ('timidity', 2.0), ('feminine', 2.0), ('sleep', 1.0), ('cheerfulness', 1.0), ('envy', 1.0), ('tourism', 1.0), ('night', 1.0), ('religion', 1.0), ('hipster', 1.0), ('worship', 1.0), ('military', 1.0), ('hiking', 1.0), ('hygiene', 1.0), ('fire', 1.0), ('sailing', 1.0), ('warmth', 1.0), ('fun', 1.0), ('joy', 1.0), ('clothing', 1.0), ('beauty', 1.0), ('monster', 1.0), ('rural', 1.0), ('anticipation', 0.0), ('pride', 0.0), ('weakness', 0.0), ('horror', 0.0), ('royalty', 0.0), ('beach', 0.0), ('blue_collar_job', 0.0), ('superhero', 0.0), ('pet', 0.0), ('medieval', 0.0), ('zest', 0.0), ('water', 0.0), ('legend', 0.0), ('neglect', 0.0), ('swimming', 0.0), ('exotic', 0.0), ('sympathy', 0.0), ('anonymity', 0.0), ('air_travel', 0.0), ('dominant_personality', 0.0), ('politeness', 0.0), ('farming', 0.0), ('disgust', 0.0), ('rage', 0.0), ('affection', 0.0), ('ugliness', 0.0), ('anger', 0.0), ('ship', 0.0), ('disappointment', 0.0), ('ocean', 0.0)]

# ecig tweets
proposal_empath_result_ecig_tweets = [('business', 9.0), ('weather', 7.0), ('work', 7.0), ('writing', 7.0), ('negative_emotion', 6.0), ('violence', 5.0), ('positive_emotion', 5.0), ('social_media', 4.0), ('speaking', 4.0), ('gain', 4.0), ('messaging', 4.0), ('friends', 4.0), ('giving', 4.0), ('play', 3.0), ('computer', 3.0), ('optimism', 3.0), ('divine', 3.0), ('driving', 3.0), ('childish', 3.0), ('celebration', 3.0), ('communication', 3.0), ('vehicle', 3.0), ('fire', 3.0), ('phone', 3.0), ('injury', 3.0), ('fun', 3.0), ('traveling', 3.0), ('economics', 3.0), ('clothing', 3.0), ('party', 3.0), ('smell', 3.0), ('pain', 3.0), ('achievement', 3.0), ('help', 2.0), ('money', 2.0), ('wedding', 2.0), ('domestic_work', 2.0), ('cheerfulness', 2.0), ('family', 2.0), ('vacation', 2.0), ('attractive', 2.0), ('masculine', 2.0), ('swearing_terms', 2.0), ('leisure', 2.0), ('banking', 2.0), ('night', 2.0), ('art', 2.0), ('home', 2.0), ('hipster', 2.0), ('surprise', 2.0), ('movement', 2.0), ('body', 2.0), ('eating', 2.0), ('restaurant', 2.0), ('trust', 2.0), ('fabric', 2.0), ('music', 2.0), ('listen', 2.0), ('warmth', 2.0), ('car', 2.0), ('technology', 2.0), ('shape_and_size', 2.0), ('plant', 2.0), ('competing', 2.0), ('children', 2.0), ('musical', 2.0), ('sleep', 1.0), ('medical_emergency', 1.0), ('occupation', 1.0), ('prison', 1.0), ('health', 1.0), ('government', 1.0), ('weakness', 1.0), ('horror', 1.0), ('suffering', 1.0), ('royalty', 1.0), ('furniture', 1.0), ('morning', 1.0), ('stealing', 1.0), ('real_estate', 1.0), ('fear', 1.0), ('pet', 1.0), ('cooking', 1.0), ('internet', 1.0), ('reading', 1.0), ('leader', 1.0), ('noise', 1.0), ('medieval', 1.0), ('healing', 1.0), ('dominant_heirarchical', 1.0), ('love', 1.0), ('hearing', 1.0), ('order', 1.0), ('hygiene', 1.0), ('fight', 1.0), ('toy', 1.0), ('farming', 1.0), ('war', 1.0), ('urban', 1.0), ('shopping', 1.0), ('sound', 1.0), ('sailing', 1.0), ('valuable', 1.0), ('youth', 1.0), ('emotional', 1.0), ('affection', 1.0), ('shame', 1.0), ('torment', 1.0), ('strength', 1.0), ('breaking', 1.0), ('animal', 1.0), ('beauty', 1.0), ('cleaning', 1.0), ('law', 1.0), ('payment', 1.0), ('liquid', 1.0), ('weapon', 1.0), ('office', 0.0), ('dance', 0.0), ('cold', 0.0), ('hate', 0.0), ('aggression', 0.0), ('envy', 0.0), ('anticipation', 0.0), ('crime', 0.0), ('pride', 0.0), ('dispute', 0.0), ('nervousness', 0.0), ('wealthy', 0.0), ('tourism', 0.0), ('school', 0.0), ('magic', 0.0), ('beach', 0.0), ('journalism', 0.0), ('exercise', 0.0), ('kill', 0.0), ('blue_collar_job', 0.0), ('ridicule', 0.0), ('college', 0.0), ('sexual', 0.0), ('irritability', 0.0), ('superhero', 0.0), ('exasperation', 0.0), ('religion', 0.0), ('worship', 0.0), ('independence', 0.0), ('zest', 0.0), ('confusion', 0.0), ('water', 0.0), ('sports', 0.0), ('death', 0.0), ('legend', 0.0), ('heroic', 0.0), ('programming', 0.0), ('military', 0.0), ('neglect', 0.0), ('swimming', 0.0), ('exotic', 0.0), ('hiking', 0.0), ('sympathy', 0.0), ('anonymity', 0.0), ('ancient', 0.0), ('deception', 0.0), ('air_travel', 0.0), ('dominant_personality', 0.0), ('politeness', 0.0), ('meeting', 0.0), ('disgust', 0.0), ('tool', 0.0), ('rage', 0.0), ('science', 0.0), ('appearance', 0.0), ('sadness', 0.0), ('joy', 0.0), ('fashion', 0.0), ('ugliness', 0.0), ('lust', 0.0), ('anger', 0.0), ('politics', 0.0), ('ship', 0.0), ('power', 0.0), ('white_collar_job', 0.0), ('terrorism', 0.0), ('disappointment', 0.0), ('poor', 0.0), ('timidity', 0.0), ('philosophy', 0.0), ('negotiate', 0.0), ('alcohol', 0.0), ('feminine', 0.0), ('monster', 0.0), ('ocean', 0.0), ('contentment', 0.0), ('rural', 0.0)]
approval_empath_result_ecig_tweets = [('eating', 11.0), ('negative_emotion', 8.0), ('friends', 8.0), ('shopping', 7.0), ('positive_emotion', 7.0), ('play', 6.0), ('death', 6.0), ('party', 6.0), ('writing', 6.0), ('money', 5.0), ('kill', 5.0), ('business', 5.0), ('cooking', 5.0), ('toy', 5.0), ('alcohol', 5.0), ('children', 5.0), ('aggression', 4.0), ('journalism', 4.0), ('computer', 4.0), ('childish', 4.0), ('internet', 4.0), ('speaking', 4.0), ('work', 4.0), ('weapon', 4.0), ('help', 3.0), ('medical_emergency', 3.0), ('optimism', 3.0), ('real_estate', 3.0), ('driving', 3.0), ('body', 3.0), ('restaurant', 3.0), ('violence', 3.0), ('communication', 3.0), ('fabric', 3.0), ('war', 3.0), ('traveling', 3.0), ('economics', 3.0), ('politics', 3.0), ('technology', 3.0), ('animal', 3.0), ('pain', 3.0), ('cleaning', 3.0), ('law', 3.0), ('liquid', 3.0), ('domestic_work', 2.0), ('cold', 2.0), ('hate', 2.0), ('occupation', 2.0), ('family', 2.0), ('crime', 2.0), ('masculine', 2.0), ('prison', 2.0), ('health', 2.0), ('dispute', 2.0), ('government', 2.0), ('swearing_terms', 2.0), ('suffering', 2.0), ('royalty', 2.0), ('wealthy', 2.0), ('school', 2.0), ('exercise', 2.0), ('home', 2.0), ('pet', 2.0), ('reading', 2.0), ('leader', 2.0), ('independence', 2.0), ('confusion', 2.0), ('sports', 2.0), ('healing', 2.0), ('heroic', 2.0), ('celebration', 2.0), ('programming', 2.0), ('dominant_heirarchical', 2.0), ('exotic', 2.0), ('love', 2.0), ('hearing', 2.0), ('weather', 2.0), ('trust', 2.0), ('deception', 2.0), ('fight', 2.0), ('meeting', 2.0), ('listen', 2.0), ('fire', 2.0), ('youth', 2.0), ('clothing', 2.0), ('strength', 2.0), ('shape_and_size', 2.0), ('white_collar_job', 2.0), ('smell', 2.0), ('beauty', 2.0), ('messaging', 2.0), ('giving', 2.0), ('office', 1.0), ('vacation', 1.0), ('attractive', 1.0), ('nervousness', 1.0), ('morning', 1.0), ('social_media', 1.0), ('blue_collar_job', 1.0), ('college', 1.0), ('divine', 1.0), ('hipster', 1.0), ('surprise', 1.0), ('medieval', 1.0), ('hiking', 1.0), ('order', 1.0), ('ancient', 1.0), ('air_travel', 1.0), ('dominant_personality', 1.0), ('music', 1.0), ('politeness', 1.0), ('tool', 1.0), ('phone', 1.0), ('gain', 1.0), ('injury', 1.0), ('sailing', 1.0), ('valuable', 1.0), ('fun', 1.0), ('emotional', 1.0), ('affection', 1.0), ('shame', 1.0), ('ship', 1.0), ('poor', 1.0), ('plant', 1.0), ('competing', 1.0), ('payment', 1.0), ('achievement', 1.0), ('monster', 1.0), ('ocean', 1.0), ('contentment', 1.0), ('musical', 1.0), ('dance', 0.0), ('wedding', 0.0), ('sleep', 0.0), ('cheerfulness', 0.0), ('envy', 0.0), ('anticipation', 0.0), ('pride', 0.0), ('weakness', 0.0), ('horror', 0.0), ('leisure', 0.0), ('tourism', 0.0), ('furniture', 0.0), ('magic', 0.0), ('beach', 0.0), ('banking', 0.0), ('night', 0.0), ('art', 0.0), ('ridicule', 0.0), ('stealing', 0.0), ('sexual', 0.0), ('fear', 0.0), ('irritability', 0.0), ('superhero', 0.0), ('exasperation', 0.0), ('religion', 0.0), ('worship', 0.0), ('movement', 0.0), ('noise', 0.0), ('zest', 0.0), ('water', 0.0), ('legend', 0.0), ('military', 0.0), ('neglect', 0.0), ('swimming', 0.0), ('sympathy', 0.0), ('hygiene', 0.0), ('anonymity', 0.0), ('vehicle', 0.0), ('farming', 0.0), ('urban', 0.0), ('disgust', 0.0), ('sound', 0.0), ('rage', 0.0), ('science', 0.0), ('appearance', 0.0), ('warmth', 0.0), ('sadness', 0.0), ('joy', 0.0), ('fashion', 0.0), ('ugliness', 0.0), ('lust', 0.0), ('torment', 0.0), ('anger', 0.0), ('car', 0.0), ('breaking', 0.0), ('power', 0.0), ('terrorism', 0.0), ('disappointment', 0.0), ('timidity', 0.0), ('philosophy', 0.0), ('negotiate', 0.0), ('feminine', 0.0), ('rural', 0.0)]
enforcement_empath_result_01_ecig_tweets = [('negative_emotion', 11.0), ('communication', 9.0), ('traveling', 9.0), ('musical', 8.0), ('business', 7.0), ('writing', 7.0), ('domestic_work', 6.0), ('home', 6.0), ('movement', 6.0), ('eating', 6.0), ('music', 6.0), ('speaking', 6.0), ('work', 6.0), ('economics', 6.0), ('positive_emotion', 6.0), ('money', 5.0), ('family', 5.0), ('vacation', 5.0), ('night', 5.0), ('optimism', 5.0), ('cooking', 5.0), ('trust', 5.0), ('fire', 5.0), ('clothing', 5.0), ('giving', 5.0), ('dance', 4.0), ('driving', 4.0), ('internet', 4.0), ('reading', 4.0), ('shopping', 4.0), ('shame', 4.0), ('party', 4.0), ('pain', 4.0), ('friends', 4.0), ('alcohol', 4.0), ('children', 4.0), ('sleep', 3.0), ('school', 3.0), ('beach', 3.0), ('computer', 3.0), ('college', 3.0), ('childish', 3.0), ('body', 3.0), ('sports', 3.0), ('celebration', 3.0), ('violence', 3.0), ('hiking', 3.0), ('order', 3.0), ('fabric', 3.0), ('phone', 3.0), ('gain', 3.0), ('valuable', 3.0), ('smell', 3.0), ('payment', 3.0), ('liquid', 3.0), ('wedding', 2.0), ('hate', 2.0), ('cheerfulness', 2.0), ('occupation', 2.0), ('attractive', 2.0), ('masculine', 2.0), ('nervousness', 2.0), ('swearing_terms', 2.0), ('suffering', 2.0), ('wealthy', 2.0), ('tourism', 2.0), ('banking', 2.0), ('social_media', 2.0), ('blue_collar_job', 2.0), ('art', 2.0), ('real_estate', 2.0), ('sexual', 2.0), ('fear', 2.0), ('pet', 2.0), ('surprise', 2.0), ('leader', 2.0), ('healing', 2.0), ('swimming', 2.0), ('love', 2.0), ('air_travel', 2.0), ('vehicle', 2.0), ('listen', 2.0), ('urban', 2.0), ('injury', 2.0), ('warmth', 2.0), ('fun', 2.0), ('emotional', 2.0), ('fashion', 2.0), ('technology', 2.0), ('shape_and_size', 2.0), ('white_collar_job', 2.0), ('animal', 2.0), ('poor', 2.0), ('cleaning', 2.0), ('achievement', 2.0), ('weapon', 2.0), ('rural', 2.0), ('cold', 1.0), ('aggression', 1.0), ('health', 1.0), ('dispute', 1.0), ('government', 1.0), ('horror', 1.0), ('leisure', 1.0), ('royalty', 1.0), ('furniture', 1.0), ('magic', 1.0), ('kill', 1.0), ('play', 1.0), ('stealing', 1.0), ('divine', 1.0), ('irritability', 1.0), ('exasperation', 1.0), ('religion', 1.0), ('hipster', 1.0), ('independence', 1.0), ('medieval', 1.0), ('confusion', 1.0), ('water', 1.0), ('death', 1.0), ('legend', 1.0), ('heroic', 1.0), ('restaurant', 1.0), ('programming', 1.0), ('hearing', 1.0), ('weather', 1.0), ('ancient', 1.0), ('deception', 1.0), ('fight', 1.0), ('toy', 1.0), ('farming', 1.0), ('war', 1.0), ('disgust', 1.0), ('sailing', 1.0), ('rage', 1.0), ('youth', 1.0), ('sadness', 1.0), ('ugliness', 1.0), ('anger', 1.0), ('ship', 1.0), ('car', 1.0), ('strength', 1.0), ('breaking', 1.0), ('disappointment', 1.0), ('plant', 1.0), ('timidity', 1.0), ('negotiate', 1.0), ('messaging', 1.0), ('law', 1.0), ('ocean', 1.0), ('contentment', 1.0), ('help', 0.0), ('office', 0.0), ('medical_emergency', 0.0), ('envy', 0.0), ('anticipation', 0.0), ('crime', 0.0), ('prison', 0.0), ('pride', 0.0), ('weakness', 0.0), ('journalism', 0.0), ('morning', 0.0), ('exercise', 0.0), ('ridicule', 0.0), ('superhero', 0.0), ('worship', 0.0), ('noise', 0.0), ('zest', 0.0), ('dominant_heirarchical', 0.0), ('military', 0.0), ('neglect', 0.0), ('exotic', 0.0), ('sympathy', 0.0), ('hygiene', 0.0), ('anonymity', 0.0), ('dominant_personality', 0.0), ('politeness', 0.0), ('meeting', 0.0), ('tool', 0.0), ('sound', 0.0), ('science', 0.0), ('appearance', 0.0), ('joy', 0.0), ('affection', 0.0), ('lust', 0.0), ('torment', 0.0), ('politics', 0.0), ('power', 0.0), ('terrorism', 0.0), ('beauty', 0.0), ('philosophy', 0.0), ('competing', 0.0), ('feminine', 0.0), ('monster', 0.0)]


In [40]:
print('Intersection in proposal month')
print()
get_same_topics(proposal_empath_result_reddit, proposal_empath_result_ecig_tweets, topN=10)
get_same_topics(proposal_empath_result_reddit, proposal_empath_result_ecig_tweets, topN=20)
get_same_topics(proposal_empath_result_reddit, proposal_empath_result_ecig_tweets, topN=30)
get_same_topics(proposal_empath_result_reddit, proposal_empath_result_ecig_tweets, topN=40)

Intersection in proposal month

- topN:  10
  - intersection:  [('negative_emotion', 232.0), ('business', 199.0), ('speaking', 195.0)]
- topN:  20
  - intersection:  [('negative_emotion', 232.0), ('business', 199.0), ('speaking', 195.0), ('positive_emotion', 117.0), ('giving', 101.0), ('work', 101.0), ('violence', 96.0)]
- topN:  30
  - intersection:  [('negative_emotion', 232.0), ('communication', 220.0), ('business', 199.0), ('speaking', 195.0), ('economics', 174.0), ('positive_emotion', 117.0), ('giving', 101.0), ('work', 101.0), ('violence', 96.0), ('social_media', 93.0), ('messaging', 93.0), ('writing', 89.0), ('optimism', 77.0)]
- topN:  40
  - intersection:  [('negative_emotion', 232.0), ('smell', 222.0), ('communication', 220.0), ('business', 199.0), ('speaking', 195.0), ('economics', 174.0), ('money', 120.0), ('positive_emotion', 117.0), ('giving', 101.0), ('work', 101.0), ('violence', 96.0), ('social_media', 93.0), ('messaging', 93.0), ('writing', 89.0), ('optimism', 77.0), (

In [35]:
print('Intersection in approval month')
print()
get_same_topics(approval_empath_result_reddit, approval_empath_result_ecig_tweets, topN=10)
get_same_topics(approval_empath_result_reddit, approval_empath_result_ecig_tweets, topN=20)
get_same_topics(approval_empath_result_reddit, approval_empath_result_ecig_tweets, topN=30)
get_same_topics(approval_empath_result_reddit, approval_empath_result_ecig_tweets, topN=40)

Intersection in approval month

- topN:  10
  - intersection:  [('negative_emotion', 204.0), ('shopping', 126.0), ('eating', 118.0), ('money', 102.0)]
- topN:  20
  - intersection:  [('negative_emotion', 204.0), ('business', 144.0), ('shopping', 126.0), ('eating', 118.0), ('money', 102.0), ('positive_emotion', 95.0)]
- topN:  30
  - intersection:  [('negative_emotion', 204.0), ('business', 144.0), ('speaking', 135.0), ('shopping', 126.0), ('eating', 118.0), ('money', 102.0), ('positive_emotion', 95.0), ('real_estate', 94.0), ('internet', 66.0), ('children', 62.0)]
- topN:  40
  - intersection:  [('negative_emotion', 204.0), ('communication', 151.0), ('business', 144.0), ('economics', 136.0), ('speaking', 135.0), ('shopping', 126.0), ('eating', 118.0), ('money', 102.0), ('positive_emotion', 95.0), ('real_estate', 94.0), ('violence', 72.0), ('internet', 66.0), ('traveling', 63.0), ('children', 62.0), ('optimism', 57.0), ('work', 57.0), ('computer', 55.0)]


In [36]:
print('Intersection in enforcement month')
print()
get_same_topics(enforcement_empath_result_01_reddit, enforcement_empath_result_01_ecig_tweets, topN=10)
get_same_topics(enforcement_empath_result_01_reddit, enforcement_empath_result_01_ecig_tweets, topN=20)
get_same_topics(enforcement_empath_result_01_reddit, enforcement_empath_result_01_ecig_tweets, topN=30)
get_same_topics(enforcement_empath_result_01_reddit, enforcement_empath_result_01_ecig_tweets, topN=40)

Intersection in enforcement month

- topN:  10
  - intersection:  [('negative_emotion', 58.0), ('communication', 38.0), ('business', 37.0)]
- topN:  20
  - intersection:  [('negative_emotion', 58.0), ('communication', 38.0), ('money', 37.0), ('speaking', 37.0), ('business', 37.0), ('economics', 34.0), ('positive_emotion', 25.0)]
- topN:  30
  - intersection:  [('negative_emotion', 58.0), ('communication', 38.0), ('money', 37.0), ('speaking', 37.0), ('business', 37.0), ('economics', 34.0), ('shopping', 30.0), ('positive_emotion', 25.0), ('giving', 24.0), ('trust', 20.0), ('internet', 19.0), ('reading', 19.0), ('family', 19.0)]
- topN:  40
  - intersection:  [('negative_emotion', 58.0), ('communication', 38.0), ('money', 37.0), ('speaking', 37.0), ('business', 37.0), ('economics', 34.0), ('shopping', 30.0), ('children', 28.0), ('positive_emotion', 25.0), ('giving', 24.0), ('alcohol', 20.0), ('trust', 20.0), ('home', 20.0), ('eating', 20.0), ('school', 19.0), ('internet', 19.0), ('reading

## LIWC

In [19]:
# reddit
proposal_liwc_result_reddit = [('function (Function Words)', 12677), ('verb (Verbs)', 3715), ('prep (Prepositions)', 3605), ('cogproc (Cognitive Processes)', 3465), ('relativ (Relativity)', 3071), ('pronoun (Pronouns)', 2909), ('focuspresent (Present Focus)', 2442), ('social (Social)', 2180), ('auxverb (Auxiliary Verbs)', 1876), ('article (Articles)', 1694), ('ipron (Impersonal Pronouns)', 1654), ('drives (Drives)', 1607), ('space (Space)', 1601), ('conj (Conjunctions)', 1579), ('adverb (Adverbs)', 1348), ('affect (Affect)', 1320), ('ppron (Personal Pronouns)', 1255), ('adj (Adjectives)', 1235), ('time (Time)', 1079), ('bio (Biological Processes)', 962), ('differ (Differentiation)', 961), ('tentat (Tentative)', 875), ('percept (Perceptual Processes)', 842), ('focuspast (Past Focus)', 814), ('work (Work)', 770), ('compare (Comparisons)', 758), ('posemo (Positive Emotions)', 742), ('cause (Causal)', 675), ('quant (Quantifiers)', 642), ('ingest (Ingest)', 600), ('insight (Insight)', 593), ('power (Power)', 574), ('negemo (Negative Emotions)', 561), ('certain (Certainty)', 471), ('you (You)', 441), ('money (Money)', 434), ('discrep (Discrepancies)', 413), ('interrog (Interrogatives)', 410), ('motion (Motion)', 385), ('they (They)', 362), ('achieve (Achievement)', 350), ('negate (Negations)', 349), ('informal (Informal Language)', 335), ('reward (Reward)', 326), ('affiliation (Affiliation)', 322), ('health (Health)', 278), ('focusfuture (Future Focus)', 271), ('leisure (Leisure)', 266), ('anger (Anger)', 256), ('i (I)', 255), ('see (See)', 204), ('risk (Risk)', 180), ('swear (Swear)', 168), ('number (Numbers)', 168), ('body (Body)', 164), ('hear (Hear)', 152), ('we (We)', 148), ('feel (Feel)', 90), ('anx (Anx)', 60), ('sexual (Sexual)', 58), ('netspeak (Netspeak)', 55), ('sad (Sad)', 54), ('male (Male)', 53), ('family (Family)', 50), ('assent (Assent)', 50), ('shehe (SheHe)', 49), ('nonflu (Nonfluencies)', 48), ('home (Home)', 39), ('female (Female)', 35), ('friend (Friends)', 28), ('death (Death)', 26), ('relig (Religion)', 18), ('filler (Filler Words)', 12)]
approval_liwc_result_reddit = [('function (Function Words)', 9880), ('verb (Verbs)', 2893), ('prep (Prepositions)', 2829), ('cogproc (Cognitive Processes)', 2559), ('relativ (Relativity)', 2552), ('pronoun (Pronouns)', 2123), ('focuspresent (Present Focus)', 1886), ('social (Social)', 1577), ('space (Space)', 1522), ('auxverb (Auxiliary Verbs)', 1480), ('drives (Drives)', 1407), ('article (Articles)', 1394), ('conj (Conjunctions)', 1208), ('ipron (Impersonal Pronouns)', 1200), ('affect (Affect)', 1087), ('adverb (Adverbs)', 1033), ('adj (Adjectives)', 993), ('ppron (Personal Pronouns)', 921), ('time (Time)', 708), ('differ (Differentiation)', 703), ('focuspast (Past Focus)', 654), ('power (Power)', 652), ('tentat (Tentative)', 593), ('work (Work)', 578), ('posemo (Positive Emotions)', 574), ('percept (Perceptual Processes)', 567), ('compare (Comparisons)', 558), ('bio (Biological Processes)', 520), ('quant (Quantifiers)', 505), ('negemo (Negative Emotions)', 501), ('cause (Causal)', 459), ('insight (Insight)', 448), ('money (Money)', 355), ('certain (Certainty)', 354), ('you (You)', 318), ('motion (Motion)', 314), ('they (They)', 313), ('discrep (Discrepancies)', 312), ('informal (Informal Language)', 292), ('ingest (Ingest)', 281), ('achieve (Achievement)', 274), ('negate (Negations)', 271), ('interrog (Interrogatives)', 265), ('anger (Anger)', 248), ('affiliation (Affiliation)', 236), ('reward (Reward)', 225), ('focusfuture (Future Focus)', 199), ('leisure (Leisure)', 191), ('health (Health)', 164), ('risk (Risk)', 161), ('number (Numbers)', 148), ('swear (Swear)', 138), ('see (See)', 131), ('i (I)', 129), ('hear (Hear)', 124), ('we (We)', 111), ('body (Body)', 93), ('feel (Feel)', 93), ('netspeak (Netspeak)', 73), ('male (Male)', 66), ('sad (Sad)', 60), ('sexual (Sexual)', 52), ('shehe (SheHe)', 50), ('assent (Assent)', 49), ('home (Home)', 40), ('anx (Anx)', 36), ('family (Family)', 29), ('relig (Religion)', 26), ('death (Death)', 25), ('friend (Friends)', 22), ('nonflu (Nonfluencies)', 22), ('female (Female)', 11), ('filler (Filler Words)', 9)]
enforcement_liwc_result_01_reddit = [('function (Function Words)', 1967), ('verb (Verbs)', 624), ('cogproc (Cognitive Processes)', 553), ('prep (Prepositions)', 548), ('relativ (Relativity)', 480), ('focuspresent (Present Focus)', 429), ('pronoun (Pronouns)', 393), ('auxverb (Auxiliary Verbs)', 342), ('drives (Drives)', 323), ('social (Social)', 318), ('article (Articles)', 262), ('conj (Conjunctions)', 246), ('space (Space)', 246), ('adverb (Adverbs)', 225), ('ipron (Impersonal Pronouns)', 221), ('affect (Affect)', 205), ('adj (Adjectives)', 191), ('time (Time)', 181), ('ppron (Personal Pronouns)', 172), ('bio (Biological Processes)', 160), ('differ (Differentiation)', 149), ('work (Work)', 145), ('power (Power)', 132), ('tentat (Tentative)', 124), ('focuspast (Past Focus)', 113), ('cause (Causal)', 112), ('negemo (Negative Emotions)', 104), ('compare (Comparisons)', 103), ('quant (Quantifiers)', 101), ('posemo (Positive Emotions)', 100), ('money (Money)', 97), ('insight (Insight)', 94), ('certain (Certainty)', 86), ('ingest (Ingest)', 86), ('health (Health)', 81), ('percept (Perceptual Processes)', 77), ('discrep (Discrepancies)', 69), ('achieve (Achievement)', 69), ('affiliation (Affiliation)', 64), ('motion (Motion)', 64), ('informal (Informal Language)', 60), ('they (They)', 58), ('negate (Negations)', 58), ('leisure (Leisure)', 57), ('focusfuture (Future Focus)', 55), ('interrog (Interrogatives)', 54), ('i (I)', 53), ('reward (Reward)', 50), ('risk (Risk)', 49), ('anger (Anger)', 42), ('body (Body)', 27), ('swear (Swear)', 25), ('number (Numbers)', 24), ('you (You)', 24), ('we (We)', 23), ('see (See)', 22), ('sad (Sad)', 19), ('netspeak (Netspeak)', 19), ('feel (Feel)', 19), ('hear (Hear)', 17), ('shehe (SheHe)', 14), ('family (Family)', 14), ('female (Female)', 13), ('anx (Anx)', 12), ('death (Death)', 9), ('nonflu (Nonfluencies)', 7), ('home (Home)', 7), ('assent (Assent)', 7), ('friend (Friends)', 7), ('male (Male)', 7), ('sexual (Sexual)', 5), ('relig (Religion)', 4)]

# ecig tweets
proposal_liwc_result_ecig_tweets = [('function (Function Words)', 338), ('prep (Prepositions)', 107), ('relativ (Relativity)', 101), ('verb (Verbs)', 86), ('pronoun (Pronouns)', 83), ('social (Social)', 70), ('focuspresent (Present Focus)', 67), ('ppron (Personal Pronouns)', 55), ('cogproc (Cognitive Processes)', 52), ('auxverb (Auxiliary Verbs)', 49), ('space (Space)', 49), ('article (Articles)', 46), ('leisure (Leisure)', 43), ('adj (Adjectives)', 42), ('drives (Drives)', 41), ('time (Time)', 39), ('conj (Conjunctions)', 33), ('percept (Perceptual Processes)', 29), ('ipron (Impersonal Pronouns)', 28), ('work (Work)', 26), ('adverb (Adverbs)', 25), ('compare (Comparisons)', 24), ('bio (Biological Processes)', 24), ('affect (Affect)', 23), ('informal (Informal Language)', 18), ('i (I)', 17), ('you (You)', 16), ('posemo (Positive Emotions)', 16), ('focuspast (Past Focus)', 16), ('see (See)', 15), ('affiliation (Affiliation)', 15), ('achieve (Achievement)', 14), ('quant (Quantifiers)', 12), ('ingest (Ingest)', 12), ('focusfuture (Future Focus)', 12), ('motion (Motion)', 12), ('tentat (Tentative)', 11), ('differ (Differentiation)', 10), ('discrep (Discrepancies)', 9), ('interrog (Interrogatives)', 9), ('we (We)', 9), ('reward (Reward)', 9), ('shehe (SheHe)', 9), ('power (Power)', 8), ('male (Male)', 8), ('insight (Insight)', 8), ('negate (Negations)', 8), ('cause (Causal)', 8), ('negemo (Negative Emotions)', 7), ('certain (Certainty)', 7), ('number (Numbers)', 6), ('hear (Hear)', 6), ('money (Money)', 6), ('body (Body)', 6), ('anger (Anger)', 6), ('female (Female)', 6), ('feel (Feel)', 5), ('they (They)', 4), ('netspeak (Netspeak)', 4), ('friend (Friends)', 4), ('swear (Swear)', 4), ('home (Home)', 3), ('health (Health)', 3), ('sexual (Sexual)', 3), ('family (Family)', 2), ('death (Death)', 2), ('assent (Assent)', 1), ('sad (Sad)', 1), ('risk (Risk)', 1), ('nonflu (Nonfluencies)', 1)]
approval_liwc_result_ecig_tweets = [('function (Function Words)', 423), ('prep (Prepositions)', 125), ('relativ (Relativity)', 119), ('verb (Verbs)', 115), ('pronoun (Pronouns)', 101), ('focuspresent (Present Focus)', 79), ('cogproc (Cognitive Processes)', 75), ('social (Social)', 68), ('article (Articles)', 68), ('drives (Drives)', 63), ('space (Space)', 62), ('ppron (Personal Pronouns)', 60), ('auxverb (Auxiliary Verbs)', 57), ('adverb (Adverbs)', 50), ('time (Time)', 43), ('leisure (Leisure)', 42), ('adj (Adjectives)', 42), ('affect (Affect)', 42), ('ipron (Impersonal Pronouns)', 41), ('conj (Conjunctions)', 41), ('bio (Biological Processes)', 36), ('i (I)', 33), ('work (Work)', 30), ('percept (Perceptual Processes)', 29), ('focuspast (Past Focus)', 28), ('posemo (Positive Emotions)', 27), ('compare (Comparisons)', 26), ('power (Power)', 24), ('tentat (Tentative)', 22), ('informal (Informal Language)', 21), ('you (You)', 20), ('differ (Differentiation)', 20), ('quant (Quantifiers)', 18), ('reward (Reward)', 17), ('motion (Motion)', 17), ('ingest (Ingest)', 17), ('netspeak (Netspeak)', 16), ('interrog (Interrogatives)', 15), ('negemo (Negative Emotions)', 15), ('cause (Causal)', 15), ('insight (Insight)', 14), ('affiliation (Affiliation)', 13), ('body (Body)', 11), ('achieve (Achievement)', 11), ('certain (Certainty)', 10), ('discrep (Discrepancies)', 9), ('male (Male)', 9), ('hear (Hear)', 9), ('see (See)', 9), ('focusfuture (Future Focus)', 9), ('money (Money)', 9), ('health (Health)', 8), ('anger (Anger)', 8), ('feel (Feel)', 5), ('risk (Risk)', 5), ('number (Numbers)', 4), ('friend (Friends)', 4), ('negate (Negations)', 4), ('swear (Swear)', 3), ('death (Death)', 3), ('they (They)', 3), ('female (Female)', 3), ('sad (Sad)', 2), ('we (We)', 2), ('shehe (SheHe)', 2), ('assent (Assent)', 2), ('family (Family)', 2), ('home (Home)', 1), ('relig (Religion)', 1)]
enforcement_liwc_result_01_ecig_tweets = [('function (Function Words)', 527), ('verb (Verbs)', 167), ('relativ (Relativity)', 154), ('prep (Prepositions)', 149), ('pronoun (Pronouns)', 145), ('cogproc (Cognitive Processes)', 109), ('focuspresent (Present Focus)', 103), ('ppron (Personal Pronouns)', 94), ('space (Space)', 83), ('social (Social)', 83), ('auxverb (Auxiliary Verbs)', 78), ('article (Articles)', 77), ('drives (Drives)', 59), ('i (I)', 56), ('affect (Affect)', 56), ('time (Time)', 53), ('focuspast (Past Focus)', 53), ('conj (Conjunctions)', 52), ('ipron (Impersonal Pronouns)', 51), ('adj (Adjectives)', 44), ('bio (Biological Processes)', 44), ('leisure (Leisure)', 43), ('differ (Differentiation)', 35), ('adverb (Adverbs)', 32), ('posemo (Positive Emotions)', 31), ('percept (Perceptual Processes)', 30), ('work (Work)', 29), ('informal (Informal Language)', 27), ('tentat (Tentative)', 24), ('ingest (Ingest)', 23), ('motion (Motion)', 22), ('reward (Reward)', 22), ('insight (Insight)', 22), ('cause (Causal)', 21), ('you (You)', 21), ('negemo (Negative Emotions)', 21), ('quant (Quantifiers)', 21), ('netspeak (Netspeak)', 20), ('discrep (Discrepancies)', 17), ('compare (Comparisons)', 17), ('achieve (Achievement)', 14), ('focusfuture (Future Focus)', 13), ('certain (Certainty)', 13), ('affiliation (Affiliation)', 13), ('body (Body)', 12), ('health (Health)', 12), ('negate (Negations)', 11), ('see (See)', 11), ('power (Power)', 11), ('money (Money)', 11), ('hear (Hear)', 9), ('male (Male)', 9), ('feel (Feel)', 8), ('number (Numbers)', 8), ('interrog (Interrogatives)', 7), ('they (They)', 7), ('female (Female)', 6), ('sad (Sad)', 6), ('anger (Anger)', 6), ('shehe (SheHe)', 5), ('we (We)', 5), ('friend (Friends)', 5), ('swear (Swear)', 5), ('home (Home)', 4), ('family (Family)', 3), ('risk (Risk)', 3), ('anx (Anx)', 2), ('assent (Assent)', 2), ('death (Death)', 2), ('sexual (Sexual)', 1)]


In [41]:
print('Intersection in proposal month')
print()
get_same_topics(proposal_liwc_result_reddit, proposal_liwc_result_ecig_tweets, topN=20)
print()
get_same_topics(proposal_liwc_result_reddit, proposal_liwc_result_ecig_tweets, topN=40)
print()
get_same_topics(proposal_liwc_result_reddit, proposal_liwc_result_ecig_tweets, topN=60)

Intersection in proposal month

- topN:  20
  - intersection:  [('function (Function Words)', 13015), ('verb (Verbs)', 3801), ('prep (Prepositions)', 3712), ('cogproc (Cognitive Processes)', 3517), ('relativ (Relativity)', 3172), ('pronoun (Pronouns)', 2992), ('focuspresent (Present Focus)', 2509), ('social (Social)', 2250), ('auxverb (Auxiliary Verbs)', 1925), ('article (Articles)', 1740), ('ipron (Impersonal Pronouns)', 1682), ('space (Space)', 1650), ('drives (Drives)', 1648), ('conj (Conjunctions)', 1612), ('ppron (Personal Pronouns)', 1310), ('adj (Adjectives)', 1277), ('time (Time)', 1118)]

- topN:  40
  - intersection:  [('function (Function Words)', 13015), ('verb (Verbs)', 3801), ('prep (Prepositions)', 3712), ('cogproc (Cognitive Processes)', 3517), ('relativ (Relativity)', 3172), ('pronoun (Pronouns)', 2992), ('focuspresent (Present Focus)', 2509), ('social (Social)', 2250), ('auxverb (Auxiliary Verbs)', 1925), ('article (Articles)', 1740), ('ipron (Impersonal Pronouns)', 1

In [42]:
print('Intersection in approval month')
print()
get_same_topics(approval_liwc_result_reddit, approval_liwc_result_ecig_tweets, topN=20)
print()
get_same_topics(approval_liwc_result_reddit, approval_liwc_result_ecig_tweets, topN=40)
print()
get_same_topics(approval_liwc_result_reddit, approval_liwc_result_ecig_tweets, topN=60)

Intersection in approval month

- topN:  20
  - intersection:  [('function (Function Words)', 10303), ('verb (Verbs)', 3008), ('prep (Prepositions)', 2954), ('relativ (Relativity)', 2671), ('cogproc (Cognitive Processes)', 2634), ('pronoun (Pronouns)', 2224), ('focuspresent (Present Focus)', 1965), ('social (Social)', 1645), ('space (Space)', 1584), ('auxverb (Auxiliary Verbs)', 1537), ('drives (Drives)', 1470), ('article (Articles)', 1462), ('conj (Conjunctions)', 1249), ('ipron (Impersonal Pronouns)', 1241), ('affect (Affect)', 1129), ('adverb (Adverbs)', 1083), ('adj (Adjectives)', 1035), ('ppron (Personal Pronouns)', 981), ('time (Time)', 751)]

- topN:  40
  - intersection:  [('function (Function Words)', 10303), ('verb (Verbs)', 3008), ('prep (Prepositions)', 2954), ('relativ (Relativity)', 2671), ('cogproc (Cognitive Processes)', 2634), ('pronoun (Pronouns)', 2224), ('focuspresent (Present Focus)', 1965), ('social (Social)', 1645), ('space (Space)', 1584), ('auxverb (Auxiliary V

In [43]:
print('Intersection in enforcement month')
print()
get_same_topics(enforcement_liwc_result_01_reddit, enforcement_liwc_result_01_ecig_tweets, topN=20)
print()
get_same_topics(enforcement_liwc_result_01_reddit, enforcement_liwc_result_01_ecig_tweets, topN=40)
print()
get_same_topics(enforcement_liwc_result_01_reddit, enforcement_liwc_result_01_ecig_tweets, topN=60)

Intersection in enforcement month

- topN:  20
  - intersection:  [('function (Function Words)', 2494), ('verb (Verbs)', 791), ('prep (Prepositions)', 697), ('cogproc (Cognitive Processes)', 662), ('relativ (Relativity)', 634), ('pronoun (Pronouns)', 538), ('focuspresent (Present Focus)', 532), ('auxverb (Auxiliary Verbs)', 420), ('social (Social)', 401), ('drives (Drives)', 382), ('article (Articles)', 339), ('space (Space)', 329), ('conj (Conjunctions)', 298), ('ipron (Impersonal Pronouns)', 272), ('ppron (Personal Pronouns)', 266), ('affect (Affect)', 261), ('adj (Adjectives)', 235), ('time (Time)', 234)]

- topN:  40
  - intersection:  [('function (Function Words)', 2494), ('verb (Verbs)', 791), ('prep (Prepositions)', 697), ('cogproc (Cognitive Processes)', 662), ('relativ (Relativity)', 634), ('pronoun (Pronouns)', 538), ('focuspresent (Present Focus)', 532), ('auxverb (Auxiliary Verbs)', 420), ('social (Social)', 401), ('drives (Drives)', 382), ('article (Articles)', 339), ('spa

## LIWC remove preposition